### FactRuEval example (Cased model), MutiHeadAttention

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings

warnings.filterwarnings("ignore")

In [2]:
import os


data_path = "/home/lis/ner/ulmfit/data/factrueval/"
train_path = os.path.join(data_path, "train_with_pos.csv")
valid_path = os.path.join(data_path, "valid_with_pos.csv")
model_dir = " /datadrive/models/multi_cased_L-12_H-768_A-12/"
init_checkpoint_pt = os.path.join("/datadrive/models/multi_cased_L-12_H-768_A-12/", "pytorch_model.bin")
bert_config_file = os.path.join("/datadrive/bert/multi_cased_L-12_H-768_A-12/", "bert_config.json")
vocab_file = os.path.join("/datadrive/bert/multi_cased_L-12_H-768_A-12/", "vocab.txt")

In [3]:
import torch
torch.cuda.set_device(1)
torch.cuda.is_available(), torch.cuda.current_device()

(True, 1)

### 1. Create dataloaders

In [5]:
from modules import NerData

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [6]:
data = NerData.create(train_path, valid_path, vocab_file)

For factrueval we use the following sample of labels:

In [7]:
print(data.label2idx)

{'<pad>': 0, '[CLS]': 1, '[SEP]': 2, 'B_O': 3, 'I_O': 4, 'B_ORG': 5, 'I_ORG': 6, 'B_LOC': 7, 'I_LOC': 8, 'B_PER': 9, 'I_PER': 10}


### 2. Create model
For creating pytorch model we need to create `NerModel` object.

In [25]:
from modules.models.models import BertBiLSTMAttnCRF

In [26]:
model = BertBiLSTMAttnCRF.create(len(data.label2idx), bert_config_file, init_checkpoint_pt, enc_hidden_dim=256)

In [27]:
model.decoder

AttnCRFDecoder(
  (attn): MultiHeadAttention(
    (attention): _MultiHeadAttention(
      (attention): ScaledDotProductAttention(
        (softmax): Softmax()
        (dropout): Dropout(p=0.5)
      )
    )
    (proj): Linear(in_features=192, out_features=256, bias=True)
    (dropout): Dropout(p=0.5)
    (layer_norm): LayerNormalization()
  )
  (linear): Linears(
    (linears): ModuleList(
      (0): Linear(in_features=256, out_features=128, bias=True)
    )
    (output_linear): Linear(in_features=128, out_features=11, bias=True)
  )
  (crf): CRF()
)

In [28]:
model.get_n_trainable_params()

1151425

### 3. Create learner

For training our pytorch model we need to create `NerLearner` object.

In [29]:
from modules import NerLearner

In [30]:
learner = NerLearner(model, data,
                     best_model_path="/datadrive/models/factrueval/exp_final_attn_cased.cpt",
                     base_lr=0.0001, lr_max=0.005, clip=5.0, use_lr_scheduler=True, sup_labels=data.id2label[5:])

INFO:root:Use lr OneCycleScheduler...


### 4. Learn your NER model
Call `learner.fit`

In [31]:
learner.fit(100, target_metric='prec')

INFO:root:Resuming train... Current epoch 0.


INFO:root:
lr after epoch: 0.004996961373853585
INFO:root:
epoch 1, average train epoch loss=7.4822



INFO:root:on epoch 0 by max_prec: 0.839
INFO:root:Saving new best model...


              precision    recall  f1-score   support

       B_ORG      0.790     0.784     0.787       259
       I_ORG      0.823     0.846     0.834      1000
       B_LOC      0.780     0.885     0.829       192
       I_LOC      0.709     0.884     0.787       303
       B_PER      0.963     0.957     0.960       188
       I_PER      0.972     0.961     0.967       649

   micro avg      0.845     0.884     0.864      2591
   macro avg      0.839     0.886     0.861      2591
weighted avg      0.851     0.884     0.866      2591



INFO:root:
lr after epoch: 0.004987808013925702
INFO:root:
epoch 2, average train epoch loss=2.1233



...

INFO:root:on epoch 53 by max_prec: 0.936


              precision    recall  f1-score   support

       B_ORG      0.827     0.865     0.845       259
       I_ORG      0.906     0.867     0.886      1000
       B_LOC      0.902     0.906     0.904       192
       I_LOC      0.886     0.871     0.879       303
       B_PER      0.995     0.963     0.978       188
       I_PER      1.000     0.966     0.983       649

   micro avg      0.924     0.902     0.913      2591
   macro avg      0.919     0.906     0.912      2591
weighted avg      0.925     0.902     0.913      2591



INFO:root:
lr after epoch: 0.0039128145031334174
INFO:root:
epoch 100, average train epoch loss=0.065772



INFO:root:on epoch 53 by max_prec: 0.936


              precision    recall  f1-score   support

       B_ORG      0.845     0.861     0.853       259
       I_ORG      0.916     0.834     0.873      1000
       B_LOC      0.919     0.885     0.902       192
       I_LOC      0.907     0.838     0.871       303
       B_PER      0.948     0.979     0.963       188
       I_PER      0.965     0.985     0.975       649

   micro avg      0.923     0.889     0.906      2591
   macro avg      0.917     0.897     0.906      2591
weighted avg      0.923     0.889     0.905      2591



### 5. Evaluate
Create new data loader from existing path.

In [32]:
from modules.data.data import get_bert_data_loader_for_predict

In [33]:
dl = get_bert_data_loader_for_predict(data_path + "valid.csv", learner)

In [34]:
learner.load_model()

In [35]:
preds = learner.predict(dl)

IOB precision

In [36]:
from modules.train.train import validate_step
print(validate_step(learner.data.valid_dl, learner.model, learner.data.id2label, learner.sup_labels))

              precision    recall  f1-score   support

       B_ORG      0.849     0.849     0.849       259
       I_ORG      0.921     0.860     0.889      1000
       B_LOC      0.933     0.865     0.897       192
       I_LOC      0.936     0.822     0.875       303
       B_PER      0.989     0.979     0.984       188
       I_PER      0.988     0.982     0.985       649

   micro avg      0.938     0.894     0.916      2591
   macro avg      0.936     0.893     0.913      2591
weighted avg      0.938     0.894     0.915      2591



Span precision

In [37]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

              precision    recall  f1-score   support

         LOC      0.865     0.802     0.832       192
         ORG      0.822     0.822     0.822       259
         PER      0.946     0.936     0.941       188

   micro avg      0.872     0.850     0.861       639
   macro avg      0.878     0.854     0.865       639
weighted avg      0.872     0.850     0.860       639

